CSV generated on databricks notebook `#232437` on Nov 26

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('disconnect_scripts.csv')
df.head()

,script_url_stemmed,script_url_domain,filter,regex
0,psonstrentie.info/cADY9C6hbq2F5_lkSgWUQz9/MUjq...,psonstrentie.info,||psonstrentie.info/*/AJyYNUToIS1N^,psonstrentie.info/.+/AJyYNUToIS1N
1,client.perimeterx.net/PXrf8vapwA/main.min.js,perimeterx.net,||perimeterx.net/*/main.min.js^,perimeterx.net/.+/main.min.js
2,api-identification.hotmart.com/id.min.js,hotmart.com,||hotmart.com/*/id.min.js^,hotmart.com/.+/id.min.js
3,sonar.semantiqo.com/c83ul/checking.js,semantiqo.com,||semantiqo.com/*/checking.js^,semantiqo.com/.+/checking.js
4,go.ero-advertising.com/loadeactrl.go,ero-advertising.com,||ero-advertising.com/*/loadeactrl.go^,ero-advertising.com/.+/loadeactrl.go


In [6]:
df.script_url_domain.nunique()

80

In [8]:
df['filter'].nunique()

487

This all matches with databricks notebook.

In [17]:
lines = []
lines.append('! Title: Fingerprinting Script Filters')
lines.append('! Description: Scripts to block')
lines.append('! Expires: 100 days')
lines.extend(df['filter'].unique())

with open('../src/treatmentScript/assets/script_list.txt', 'w') as f:
    f.write('\n'.join(lines))